# Setup

You might need to run the cell below to setup data required by the Luminesce commands.


In [ ]:
# Import general modules
import logging
import os
import pathlib


import lusid_notifications
from fbnsdkutilities import ApiClientFactory, ApiConfigurationLoader
import lusid_notifications.models as ln_models
import json

# Import LUSID Drive modules
from lusid_drive.utilities import ApiClientFactory as DriveApiClientFactory

from runner import create_temp_folder, add_file_to_temp_folder

# Create loggers
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

unique_folder_name = "recon-data-bootstrap"


def portfolio_setup_main(api_factory, data_dir):

    # Create a new temp folder
    logger.debug(f"Create a new folder: {unique_folder_name}")
    create_temp_folder(api_factory, unique_folder_name)

    for file in data_dir.iterdir():

        file_name = os.path.basename(file)

        if file_name.endswith(("csv", "xlsx")):

            logger.debug(f"Adding the following file to folder: {file}")

            add_file_to_temp_folder(api_factory, file, unique_folder_name)


def create_manual_email_subscription(scope, code):

    try:
        subs_api.create_subscription(
            create_subscription=ln_models.CreateSubscription(
                id=ln_models.ResourceId(scope=scope, code=code),
                display_name="Subscription for holdings rec result",
                description="Subscription for holdings rec result",
                status="Active",
                matching_pattern=ln_models.MatchingPattern(event_type="Manual",
                                                           filter=f"Message eq '{code}'"))
        )

    except lusid_notifications.ApiException as e:
        if json.loads(e.body)["code"] == 711:
            logging.warning( json.loads(e.body)["detail"])
        else:
            raise


def notifications_setup():

    notification_scope = "ibor"
    notification_codes = ["HoldingReconFailed", "HoldingReconPassed"]

    for code in notification_codes:

        create_manual_email_subscription(scope=notification_scope, code=code)
        create_email_notification(scope=notification_scope, code=code)


def create_email_notification(scope, code):

    """
    IMPORTANT:
    This function is used to create Email notifications.
    We have disabled the function for CI/CD.
    To enable this function, add an email as a param under the
    CreateEmailNotification model
    """

    # Notifications don't have a unique id that we can set, so instead search by their description
    description = 'Email for results of holding recon'

    notifications = notifications_api.list_notifications(scope=scope, code=code)
    existing_notification = [i.description for i in notifications.values if i.description == description]

    if(not existing_notification):

        notifications_api.create_email_notification(
                scope=scope, code=code,
                create_email_notification=ln_models.CreateEmailNotification(
                    description=description,
                    subject=f"Holding reconcilation: {code}",
                    plain_text_body=f"""
                    Holdings reconcilation has finished with status {code}.
                    Please see results in the following LUSID drive directory:
                    <Enter LUSID Drive location here>
                    """,
                    email_address_to=["steve.collie@finbourne.com"] # IMPORTANT - update this email
            )
        )

    else:
        logger.warning(f"Creation skipped because there are already notifications attached to subscription scope:{scope} code:{code}")

if __name__ == "__main__":

    data_dir = pathlib.Path(__file__).parent.resolve()

    # Create API factory
    secrets_file = (
        pathlib.Path(__file__)
        .parent.parent.parent.parent.parent.joinpath("runner", "secrets.json")
        .resolve()
    )

    # Notifications API
    config = ApiConfigurationLoader.load(lusid_notifications, secrets_file)
    naf = ApiClientFactory(lusid_notifications, token=config.access_token, api_secrets_filename=secrets_file)
    subs_api = naf.build(lusid_notifications.api.SubscriptionsApi)
    notifications_api = naf.build(lusid_notifications.api.NotificationsApi)

    # Drive APIs
    api_factory = DriveApiClientFactory(token=config.access_token, api_secrets_filename=secrets_file)

    # Setup porfolio data in Drive
    portfolio_setup_main(api_factory, data_dir)

    # Create email notifications

    ### NOTE ###
    # Uncomment to run.
    # This will attempt to generate emails.

    #notifications_setup()


# Run a reconciliation

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create transaction portfolio

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- 1. In this query, we create two Transaction Portfolios in LUSID
-- ===============================================================

-- Defining scope and code variables
@@portfolioScope =

select 'LuminesceReconExample';

@@portfolioCode1 =

select 'UkEquityTracker';

@@portfolioCode2 =

select 'UkEquityActive';

@@base_currency =

select 'GBP';

@@created_date = select #2000-01-01#;

-- Define the portfolio data
@create_portfolio =

select 'Transaction' as PortfolioType, @@portfolioScope as PortfolioScope, @@portfolioCode1 as
   PortfolioCode, @@portfolioCode1 as DisplayName, '' as Description, @@created_date as Created,
   '' as SubHoldingKeys, @@base_currency as BaseCurrency

union all

values (
   'Transaction', @@portfolioScope, @@portfolioCode2, @@portfolioCode2, '', @@created_date, '',
   @@base_currency
   );

-- Upload the portfolio into LUSID
@response_create_portfolio =

select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

select *
from @response_create_portfolio;



#### Step 2: Create instruments

In [ ]:
%%luminesce

-- =====================================================
-- Description:
-- 1. In this query, we  load instruments from an Excel
-- file into LUSID
-- =====================================================

@@file_date =

select strftime('20221101');

@instruments_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=instruments
--addFileName
enduse;

-- Run instruments transformation
@instruments_for_upload =

select Ticker, Name as DisplayName, ISIN as Isin, 'EQ' || Isin as ClientInternal, SEDOL as Sedol,
   Currency as DomCcy, 'Equities' as AssetClass, 'Equities' as SimpleInstrumentType
from @instruments_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @instruments_for_upload;



#### Step 3: Create holdings

In [ ]:
%%luminesce

-- =====================================================
-- Description:
-- 1. In this query, we load holdings into LUSID for the
-- two portfolios.
-- =====================================================

@@file_date =

select strftime('20221101');

@@portfolioScope =

select 'LuminesceReconExample';

@holdings_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=lusid_holdings
--addFileName
enduse;

-- Run holdings transformation
@holdings_for_upload =

select portfolio as [PortfolioCode], @@portfolioScope as [PortfolioScope], holding_date as
   [EffectiveAt], 'EQ' || InstrumentId as [ClientInternal], units as [Units], Currency as
   CostCurrency
from @holdings_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Portfolio.Holding.Writer
where ToWrite = @holdings_for_upload;



#### Step 4: Create reconciliation view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we create a view to run the reconciliation
-- ============================================================


@recon_view =

use Sys.Admin.SetupView
--provider=Test.Example.HoldingsRecon

--parameters
file_name,Text,equity_holdings_20221101.xlsx,true
portfolio,Text,UkEquityTracker,true
scope,Text,LuminesceReconExample,true
recon_date,Date,2022-03-01,true
----

@@file_name = select #PARAMETERVALUE(file_name);
@@portfolio = select #PARAMETERVALUE(portfolio);
@@scope = select #PARAMETERVALUE(scope);
@@recon_date = select #PARAMETERVALUE(recon_date);


@ext_holdings_from_excel =

use Drive.Excel with @@file_name
--file=/luminesce-examples/{@@file_name}
--worksheet=ext_holdings
--addFileName
enduse;

-- Get LUSID holdings

@holdings_from_lusid = select
h.PortfolioCode,
i.DisplayName as [InstrumentName],
i.LusidInstrumentId,
i.ClientInternal,
i.Isin,
h.HoldingType,
h.Units
from Lusid.Portfolio.Holding h
join Lusid.Instrument i on (i.LusidInstrumentId = h.LusidInstrumentId)
where h.PortfolioScope = @@scope
and h.PortfolioCode = @@portfolio
and h.EffectiveAt = @@recon_date
order by h.HoldingType desc;

-- Run a reconciliation

select
l.PortfolioCode as [Left_PortfolioCode],
l.InstrumentName as [Left_InstrumentName],
l.Isin as [Left_Isin],
l.HoldingType as [Left_HoldingType],
l.Units as [Left_Units],
r.Units as [Right_Units],
(r.Units - l.Units) as [Units_Diff],
round(((r.Units *1.0) / l.Units) -1 ,5) as [Units_Diff_Pct]
from @holdings_from_lusid l
join @ext_holdings_from_excel r on (l.ClientInternal = 'EQ' || r.InstrumentId
and r.portfolio = @@portfolio);

enduse;


-- wait for view/provider to be created on the grid

select * from @recon_view wait 20;



#### Step 5: Run recon with notifications

In [ ]:
%%luminesce

-- =====================================================
-- Description:
-- 1. In this query, we create event notifications from
-- the reconciliation results
-- =====================================================

-- define strings for a pass and fail reconciliation

@@failed_event =

select 'HoldingReconFailed';

@@passed_event =

select 'HoldingReconPassed';

-- Count the number of breaks from thr reconciliation
-- We use an absolute count to assess if there are
-- any breaks or not

@recon_result =

select sum(abs(Units_Diff)) as [Breaks]
from Test.Example.HoldingsRecon
where portfolio = 'UkEquityActive';

-- If there are breaks, we generate a failed event
-- If there are no breaks, we generate a passed event

@event =

select iif(Breaks = 0, @@passed_event, @@failed_event) as [EventToGenerate]
from @recon_result;

-- Generate the event in LUSID

@@event_to_generate =

select EventToGenerate
from @event;

@event_details =

select 'Manual' as [EventType], @@event_to_generate as [Message], ('Holdings recon: ' || @@event_to_generate
      ) as [Subject], date () as [EventTime];

select *
from Notification.Event.Writer
where ToWrite = @event_details;
